In [49]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
keras = tf.keras
layers = keras.layers
models = keras.models
optimizers = keras.optimizers

In [50]:
traffic_data = pd.read_csv('datasets/traffic_data.csv', index_col=0)

In [51]:
traffic_data.tail()

,day,minute,hour,second,type
98042,Sunday,59,17,51,no_traffic
98043,Sunday,0,18,0,no_traffic
98044,Sunday,0,16,24,no_traffic
98045,Sunday,0,16,35,no_traffic
98047,Sunday,1,16,28,no_traffic


In [52]:
traffic_data['c_type'] = traffic_data['type'].apply(lambda x: 1 if x == 'traffic' else 0)

In [53]:
traffic_data = traffic_data.join(pd.get_dummies(traffic_data['day'])*1)
traffic_data.drop(columns="second")
traffic_data.dtypes

day          object
minute        int64
hour          int64
second        int64
type         object
c_type        int64
Friday        int64
Monday        int64
Saturday      int64
Sunday        int64
Thursday      int64
Tuesday       int64
Wednesday     int64
dtype: object

In [54]:
training_dataset = traffic_data.sample(frac=0.8)
testing_dataset = traffic_data[~traffic_data.index.isin(training_dataset.index)]

In [ ]:
input_columns = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
    "hour",
    "minute"
]


In [56]:
traffic_model = models.Sequential([
    layers.Dense(32, input_dim=len(input_columns), activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(1, activation='sigmoid')])

traffic_model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(), metrics=['accuracy'])

traffic_model.summary()

c:\Users\yoann\Documents\Projet AI\introduction-to-deep-learning\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 32)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,409 (5.50 KB)

 Trainable params: 1,409 (5.50 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
history_traffic_model = traffic_model.fit(training_dataset[input_columns],training_dataset[["c_type"]],epochs=30, validation_split=0.1, batch_size=100)

Epoch 1/30
617/617 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5684 - loss: 0.8667 - val_accuracy: 0.7108 - val_loss: 0.4950
Epoch 2/30
617/617 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6773 - loss: 0.5147 - val_accuracy: 0.7686 - val_loss: 0.4762
Epoch 3/30
617/617 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7086 - loss: 0.4861 - val_accuracy: 0.7342 - val_loss: 0.4611
Epoch 4/30
617/617 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7311 - loss: 0.4704 - val_accuracy: 0.7895 - val_loss: 0.4474
Epoch 5/30
617/617 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7532 - loss: 0.4519 - val_accuracy: 0.7437 - val_loss: 0.4248
Epoch 6/30
617/617 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7701 - loss: 0.4292 - val_accuracy: 0.7943 - val_loss: 0.4016
Epoch 7/30
617/617 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7807 - loss: 0.4050 - val_accuracy: 0.7961 - val_loss: 0.3739
Epoch 8/30
617/617 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7866 - loss: 0.3842 - val_accuracy: 0.

In [59]:
test_loss, test_acc = traffic_model.evaluate(
    testing_dataset[input_columns], 
    testing_dataset['c_type']
)

print(f"Evaluation result on Test Data : Loss = {test_loss}, accuracy = {test_acc}")

536/536 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9969 - loss: 0.1060
Evaluation result on Test Data : Loss = 0.07938938587903976, accuracy = 0.9980149269104004
